# Day 4: Giant Squid
https://adventofcode.com/2021/day/4

In [2]:
// https://stackoverflow.com/a/35820003

use std::{
    fs::File,
    io::{self, BufRead, BufReader},
    path::Path,
};

fn lines_from_file(filename: impl AsRef<Path>) -> io::Result<Vec<String>> {
    BufReader::new(File::open(filename)?).lines().collect()
}

let input_lines = lines_from_file("input/day04.txt").unwrap();

Implement functions for parsing a single bingo board, and for parsing all bingo boards (which start after the first input line).

In [3]:
fn parse_board(board: Vec<&str>) -> Vec<Vec<Option<i32>>> {
    board
        .iter()
        .map(|line| line.split_whitespace()
                        .map(|s| s.parse::<i32>().unwrap())
                        .map(|n| Some(n))
                        .collect())
        .collect()
}

fn bingo_boards(lines: &Vec<String>) -> Vec<Vec<Vec<Option<i32>>>> {
    lines[1..]
    .chunks(6)
    .map(|chunk| parse_board(chunk.iter().skip(1).map(|s| s.as_str()).collect::<Vec<&str>>()))
    .collect()
}

Functions for determining if a board is finished, and what the value of a board is.

In [4]:
fn board_finished(board: &Vec<Vec<Option<i32>>>) -> bool {
    board.iter().any(|row| row.iter().all(|n| n.is_none()))
    ||
    (0..board[0].len()).any(|i| board.iter().map(|row| row[i]).all(|n| n.is_none()))
}

In [5]:
fn board_value(board: &Vec<Vec<Option<i32>>>) -> i32 {
    board.iter().flatten().map(|n| n.unwrap_or(0)).sum()
}

The funcion `draw_number()` applies a drawn number to all given boards, and returns a `Vec` containing a tuple for all boards which are finished in this turn.

The result tuple consists of
* the index of the board in the `Vec` of all boards,
* the number which was drawn such that the board was finished,
* the value of the board.

In [6]:
use std::collections::HashSet;

fn draw_number(boards: &mut Vec<Vec<Vec<Option<i32>>>>, n: i32) -> Option<Vec<(usize, i32, i32)>> {
    let already_finished: HashSet<usize> = boards.iter()
        .enumerate()
        .filter(|(i, board)| board_finished(board))
        .map(|(i, _board)| i)
        .collect();
    
    for board in boards.iter_mut() {
        for row in board.iter_mut() {
            for number in row.iter_mut() {
                if *number == Some(n) {
                    *number = None
                }
            }
        }
    }
    
    let finished_now: Vec<(usize, i32, i32)> = boards.iter()
        .enumerate()
        .filter(|(i, board)| !already_finished.contains(i) && board_finished(board))
        .map(|(i, board)| (i, n, board_value(board)))
        .collect();
    
    Some(finished_now)
}

The function `ordered_finished_boards` returns a `Vec` which contains the result tuples (see above) for all finished boards in the order in which they were finished.

***Note:*** the function assumes that no boards remain unfinished.

In [7]:
fn ordered_finished_boards(lines: &Vec<String>) -> Vec<(usize, i32, i32)> {
    let numbers_drawn = lines[0].split(",").map(|s| s.parse().unwrap()).collect::<Vec<i32>>();
    let mut input_boards = bingo_boards(&lines);
    let count = input_boards.len();
    
    numbers_drawn.iter()
        .scan(input_boards, |boards, &number| draw_number(boards, number))
        .flatten()
        .take(count)
        .collect()
}

Verify the algorithm with the given test input.

In [8]:
let test_input: Vec<String> = "7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7".lines().map(|s| s.to_string()).collect();

In [9]:
ordered_finished_boards(&test_input)

[(2, 24, 188), (0, 16, 137), (1, 13, 148)]

To simplify interpreting the results, add a function that describes the finishing move for a board.

In [10]:
fn describe_move(info: (usize, i32, i32)) -> String {
    format!("Board number {} was finished when {} was drawn. The final score is {}", info.0, info.1, info.1 * info.2)
}

Verify given results for part 1 (first finished board):

In [11]:
describe_move(ordered_finished_boards(&test_input)[0])

"Board number 2 was finished when 24 was drawn. The final score is 4512"

And part 2 (last finished board):

In [12]:
describe_move(*ordered_finished_boards(&test_input).last().unwrap())

"Board number 1 was finished when 13 was drawn. The final score is 1924"

Determine the result for part 1 (first finished board):

In [13]:
describe_move(ordered_finished_boards(&input_lines)[0])

"Board number 81 was finished when 14 was drawn. The final score is 8442"

And part 2 (last finished board):

In [14]:
describe_move(*ordered_finished_boards(&input_lines).last().unwrap())

"Board number 68 was finished when 15 was drawn. The final score is 4590"